## Compute additional values

1. Average all green manufacturing across all Facilities for each day
2. Compute net power use
3. Compute efficiency

In [ ]:
df = spark.sql("SELECT * FROM hive_metastore.default.greenmanufacturing")

In [ ]:
from pyspark.sql.functions import col, avg


avg_df = df.groupBy("TS").agg(
    avg(col("AmtPowerUse")).alias("AvgAmtPowerUse"),
    avg(col("AmtPowerReturnedToGrid")).alias("AvgAmtPowerReturnedToGrid"),
    avg(col("AmtProductOutput")).alias("AvgAmtProductOutput")
)

avg_df.show()

In [ ]:
from pyspark.sql.functions import coalesce

net_power_df = avg_df.withColumn(
    "AvgAmtNetPowerConsumption",
    col("AvgAmtPowerUse") - coalesce(col("AvgAmtPowerReturnedToGrid"), col("AvgAmtPowerUse") * 0)
)

In [ ]:
efficiency_df = net_power_df.withColumn(
    "Efficiency",
    col("AvgAmtProductOutput") / col("AvgAmtNetPowerConsumption")
)

efficiency_df.show()

In [ ]:
# for the demo we override the table, in reality you would likely `append` new data

efficiency_df.write.mode("overwrite").saveAsTable("hive_metastore.default.greenefficiency")